## Filter AIS to Potential COLREGS Interactions

COLREGS interactions predominantly occur when two ships are within sight of one another. Asking the question, "given this ship and time, what other ships are visible by this ship" is difficult when looking at millions of ships. We developed a methodology to aggregate AIS data points into larger time intervals, create geometries to represent the ships path, and quickly check overlapping gemetries. This methodology allows to filter the potential encounters down quickly for additional data analysis to be completed in a resonable amount of time.

In [425]:
import psycopg2
import sys
import os
import pandas as pd
import time

## Credentials

In [426]:
HOST="htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com"
DATABASE="htm_redhorse_postgres"
USER="westinn"
PASSWORD="rootroot"

## Connect PostGis

In [433]:
#Define our connection string
conn_string = "host='"+ HOST + "' dbname='" + DATABASE +"' user='" + USER + "' password='" + PASSWORD +"'"

# NOTE: connection should be closed after the query
# if sql query fails, need to rollback or no function occurs

# run single query and return the rows
def execute(query):
    try:
        conn = psycopg2.connect(conn_string)
        cursor = conn.cursor()
        print("Query:")
        print(query)
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        return rows
    except Exception as err:
        print("Error:")
        print(err)
        cursor.execute("rollback")
        conn.close()
        

Connecting to database
	->host='htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com' dbname='htm_redhorse_postgres' user='westinn' password='rootroot'


Indexing was done within PostGresql GUI

In [428]:
## helper functions
# get all csvs in directory
def get_csvs(directory):
    csv_files = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv = os.path.join(subdir,file)
                csv_files.append(csv)
                print(file)
    return csv_files

# creates polygon rectangle using min/max lat/lon
def create_polygon(min_LAT, max_LAT, min_LON, max_LON):
    polygon_string = "LINESTRING({})"
    polygon_points = "{min_LON} {min_LAT}, {min_LON} {max_LAT}, {max_LON} {max_LAT}, {max_LON} {min_LAT}, {min_LON} {min_LAT}"
    polygon_points = polygon_points.format(min_LAT=min_LAT, max_LAT=max_LAT, min_LON=min_LON, max_LON=max_LON)
    polygon_string = polygon_string.format(polygon_points)
    return polygon_string

# creates line from points in order to create buffer around line
def create_linestring(points_arry): # [[lon, lat], [lon, lat]]
    line_string = "LINESTRING({})"
    line_points = ""
    for idx in range(0,len(points_arry)):
        line_points += "{} {}, ".format(points_arry[idx][0], points_arry[idx][1])
    line_points = line_points[:-2]
    line_string = line_string.format(line_points)
    return line_string

## Upload AIS to PostGresql

AIS data is stored in Postgresql, a relational database, with the Postgis add-on that handles GIS related data extreamly well.

In [116]:
#directory = r'C:\Users\DennisSilva\Documents\HTM\data\upload'
directory = r'C:\Users\DennisSilva\Documents\HTM\data\AIS_2017_01_Zone17\AIS_ASCII_by_UTM_Month\2017'
csv_files = get_csvs(directory)

copy_sql = """
           COPY ais FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
for csv in csv_files:
    with open(csv, 'r') as f:
        cursor.copy_expert(sql=copy_sql, file=f)
        conn.commit()

AIS_2017_01_Zone17.csv


## Query Unique MMSI

All unique MMSI are queried to iterate over the entire dataset. In practice, one would have a particular ship in mind when processing the data.

In [439]:
# get distinct MMSI
query = """
SELECT DISTINCT "MMSI" FROM ais
"""
mmsi_unique_arry = execute(query)
print(mmsi_unique_arry)

Query:

SELECT DISTINCT "MMSI" FROM ais

[(378111841,), (367602140,), (319060200,), (316029994,), (367610560,), (366766840,), (338235061,), (367306040,), (367733050,), (367667720,), (338923000,), (367742880,), (257366000,), (367412560,), (538005776,), (367343230,), (636015412,), (367135170,), (367115360,), (355833000,), (319999000,), (366960160,), (367115580,), (538003095,), (319887000,), (338131333,), (316023269,), (339303000,), (338133032,), (367761890,), (477848500,), (565174000,), (367719010,), (369970042,), (319420000,), (319370000,), (367501020,), (367696730,), (367014880,), (366919110,), (258251000,), (378112112,), (246456000,), (338017000,), (367658180,), (338205826,), (368922000,), (636015295,), (338209433,), (371583000,), (416481000,), (224517240,), (367688630,), (338192290,), (538070899,), (338004587,), (338183198,), (338178129,), (366836950,), (367541650,), (257325000,), (367701880,), (369970517,), (367595560,), (636017280,), (636016606,), (316023808,), (319767000,), (36755

## Analysis Table

The raw AIS data is separated by MMSI and grouped by hour intervals (ie the path of a ship is aggregated into one row during a 1 hour interval). Analysis table reflects the range of positions, speeds, etc. including a geometry/polygon object representing the area the ship traveled in that hour. 

The geometry for each row can either be calculated by simple rectangles using min/max LON/LAT or complex shapes via adding a 4 nautical mile buffer around the line path of the ship.

In [ ]:
failed_mmsi = []

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
start = time.time()

for idx,mmsi_unique in enumerate(mmsi_unique_arry): # iterate through each mmsi
    try:
        if idx < 528:
            continue
        
        tup = [] # holds analysis rows to upload in batch
        
        mmsi = int(mmsi_unique[0])

        query = """
        SELECT * from ais
        WHERE "MMSI"={}
        """.format(mmsi) 

        df = pd.io.sql.read_sql_query(query, conn) # get all rows with mmsi

        grouped = df.groupby(pd.Grouper(key='BaseDate', freq='1H')) # group by hour

        for key, df_group in grouped: # for each group, make one row for analysis table
            if not df_group.empty and df_group.shape[0] > 1:
                MMSI = int(df_group['MMSI'].min())
                min_LAT = df_group['LAT'].min()
                max_LAT = df_group['LAT'].max()
                min_LON = df_group['LON'].min()
                max_LON = df_group['LON'].max()
                min_BaseDate = df_group['BaseDate'].min()
                max_BaseDate = df_group['BaseDate'].max()
                min_hour_BaseDate = df_group['BaseDate'].min().replace(microsecond=0,second=0,minute=0)
                min_SOG = df_group['SOG'].min()
                max_SOG = df_group['SOG'].max()
                
                df_group = df_group.sort_values(by="BaseDate")
                points_arry = [list(t) for t in zip(df_group['LON'], df_group['LAT'])]
                geometry = create_linestring(points_arry) # line with buffer
                #geometry = create_polygon(min_LAT, max_LAT, min_LON, max_LON) # rectangle of min/max
                
                COG_arry = list(df_group['COG'])
                SOG_arry = list(df_group['SOG'])

                new_row = (MMSI, 
                           min_LAT, max_LAT, 
                           min_LON, max_LON, 
                           min_BaseDate, max_BaseDate, 
                           min_hour_BaseDate,
                           min_SOG, max_SOG, 
                           geometry,
                           points_arry,
                           COG_arry,
                           SOG_arry
                          )

                tup.append(new_row)

        # rectangle polygon
        #cursor.executemany("INSERT INTO analysis VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Buffer(ST_GeogFromText(%s), 4630)::geometry, %s, %s, %s)", tup)
        
        # buffer around line (path) polygon, # 4 nautical miles is 7408 meters
        #cursor.executemany("INSERT INTO analysis VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Polygon(ST_GeomFromText(%s),4326), %s)", tup)
        
        # potential function to decrease inserstion time https://stackoverflow.com/questions/8134602/psycopg2-insert-multiple-rows-with-one-query
        args_str = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Buffer(ST_GeogFromText(%s), 4630)::geometry, %s, %s, %s)", x).decode("utf-8") for x in tup)
        cursor.execute("INSERT INTO analysis VALUES " + args_str) 
        
        conn.commit()
        
        print(mmsi)
        print(str(idx) + " / " + str(len(mmsi_unique_arry)))
        print(round((idx+0.000)/len(mmsi_unique_arry),4))
        print(time.time() - start)
        print()
        
    except Exception as err:
        failed_mmsi.append(mmsi)
        cursor.execute("rollback")
        print(mmsi)
        print("Error: {}".format(err))
        print()
        
conn.close()

352843000
528 / 6282
0.084
14.173079490661621

367184740
529 / 6282
0.0842
28.415857791900635

338161864
530 / 6282
0.0844
29.007230758666992

538071151
531 / 6282
0.0845
36.375335931777954

367592820
532 / 6282
0.0847
36.45485067367554

376313000
533 / 6282
0.0848
50.406532764434814

367601960
534 / 6282
0.085
50.4739248752594

503001530
535 / 6282
0.0852
61.191368103027344

367485560
536 / 6282
0.0853
68.36659383773804

319706000
537 / 6282
0.0855
74.21398901939392

235059127
538 / 6282
0.0856
74.41502976417542

316021208
539 / 6282
0.0858
74.50647568702698

369970156
540 / 6282
0.086
74.63999247550964

377908000
541 / 6282
0.0861
80.77531719207764

367097210
542 / 6282
0.0863
81.59368062019348

338072697
543 / 6282
0.0864
97.805349111557

367291000
544 / 6282
0.0866
98.60079789161682

338055282
545 / 6282
0.0868
99.05548429489136

247282900
546 / 6282
0.0869
100.39238905906677

367152280
547 / 6282
0.0871
102.69241690635681

367404290
548 / 6282
0.0872
102.82672071456909

338227991


538003905
691 / 6282
0.11
793.8888578414917

367397760
692 / 6282
0.1102
802.8039634227753

338163379
693 / 6282
0.1103
803.1839981079102

338109254
694 / 6282
0.1105
804.1659622192383

338124913
695 / 6282
0.1106
811.6699657440186

338203000
696 / 6282
0.1108
822.950962305069

941001036
697 / 6282
0.111
823.0579991340637

367755660
698 / 6282
0.1111
823.1129863262177

319052800
699 / 6282
0.1113
824.0149652957916

305939000
700 / 6282
0.1114
824.1509962081909

367499850
701 / 6282
0.1116
824.2129602432251

367763020
702 / 6282
0.1117
826.8409790992737

338159986
703 / 6282
0.1119
827.4579658508301

338173004
704 / 6282
0.1121
831.9809625148773

339702000
705 / 6282
0.1122
874.9189615249634

367027140
706 / 6282
0.1124
897.0559604167938

338191971
707 / 6282
0.1125
897.9609611034393

367568270
Error: syntax error at end of input
LINE 1: INSERT INTO analysis VALUES 
                                    ^


338141862
709 / 6282
0.1129
898.5859620571136

538070316
710 / 6282
0.113
901.7169

## Check to see if Analysis Table is populated

In [461]:
query = """
SELECT * FROM analysis LIMIT 5
"""

rows = execute(query)
print(rows)

Query:

SELECT * FROM analysis LIMIT 5

[(378111841, Decimal('26.11089'), Decimal('26.1109'), Decimal('-80.1224'), Decimal('-80.12238'), datetime.datetime(2017, 1, 24, 14, 41, 31), datetime.datetime(2017, 1, 24, 14, 54, 32), datetime.datetime(2017, 1, 24, 14, 0), Decimal('0.0'), Decimal('0.0'), '0103000020E610000001000000230000005FE3AFBA440954C0EC326C7AC0253A4007765728BC0854C03D47AE6F95263A4071DB69B22A0854C0D064891406273A407B135BF1950754C03351D3130E273A40D820249E030754C0DF0DF21EAD263A40C889BA59790654C09F3BDEF0E6253A4020AA7875FC0554C028504429C3243A40AF47A2BE910554C00D1A36014D233A40633A104F3D0554C00A765DDC92213A401D42D864020554C050AF01BBA51F3A405CDF7D42E30454C090B96492981D3A401F86E118E10454C0B150F8917F1B3A4031B3C0FBFB0454C0BBADA35C6F193A40D18A2FE1320554C05870C23D7C173A409B0E08AC830554C0BE25A961B9153A40CBE9DF40EB0554C07011531938143A405A4AB0A4650654C06B066A3007133A4014498EF8650654C04203B18806133A40567DDB77EE0654C01A4063B431123A403502E2D47F0754C05888A420C1113A40E644F87A140854C017980320B9113